In [1]:
import requests # to get the HTML
from bs4 import BeautifulSoup # to get parse the text
from selenium.webdriver.common.by import By
from selenium import webdriver
import chromedriver_binary
import numpy as np

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
url = 'https://www.jobs.bg/front_job_search.php?add_sh=1&from_hp=1'

In [3]:
r = requests.get(url)

In [4]:
soup = BeautifulSoup(r.text, 'html.parser')

In [5]:
soup

 <!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<title>JOBS.BG - Предложения за работа от водещи компании в България</title>
<base href="https://www.jobs.bg/"/>
<meta content="default-src 'self' 'unsafe-inline' 'unsafe-eval' http://*.jobs.bg https://*.jobs.bg wss://im.jobs.bg https://i.newsroom.bg https://*.youtube.com https://*.youtube-nocookie.com https://*.googleapis.com https://*.gstatic.com; img-src 'self' data: https://*.jobs.bg wss://im.jobs.bg https://i.newsroom.bg https://*.youtube.com https://*.youtube-nocookie.com https://*.googleapis.com https://*.gstatic.com;" http-equiv="Content-Security-Policy"/>
<link href="https://fonts.googleapis.com/css?family=Roboto:300,400,500" rel="stylesheet"/>
<meta content="Предложения за работа от водещи компании в България" name="description"/>
<link href="https://www.jobs.bg/front_job_search.php?add_sh=1&amp;from_hp=

In [6]:
driver = webdriver.Chrome()
driver.get(url)

In [7]:
html = driver.execute_script("return document.documentElement.outerHTML")

In [8]:
sel_soup = BeautifulSoup(html, 'html.parser')

In [9]:
sel_soup

<html xmlns="http://www.w3.org/1999/xhtml"><head>
<title>JOBS.BG - Предложения за работа от водещи компании в България</title>
<base href="https://www.jobs.bg/"/>
<meta content="default-src 'self' 'unsafe-inline' 'unsafe-eval' http://*.jobs.bg https://*.jobs.bg wss://im.jobs.bg https://i.newsroom.bg https://*.youtube.com https://*.youtube-nocookie.com https://*.googleapis.com https://*.gstatic.com; img-src 'self' data: https://*.jobs.bg wss://im.jobs.bg https://i.newsroom.bg https://*.youtube.com https://*.youtube-nocookie.com https://*.googleapis.com https://*.gstatic.com;" http-equiv="Content-Security-Policy"/>
<link href="https://fonts.googleapis.com/css?family=Roboto:300,400,500" rel="stylesheet"/>
<meta content="Предложения за работа от водещи компании в България" name="description"/>
<link href="https://www.jobs.bg/front_job_search.php?add_sh=1&amp;from_hp=1&amp;" rel="canonical"/>
<link href="/front_job_search.php?frompage=15&amp;add_sh=1&amp;from_hp=1" rel="next"/>
<meta conten

In [10]:
print(sel_soup.findAll('title'))

[<title>JOBS.BG - Предложения за работа от водещи компании в България</title>]


### Postings (Details) 
This approach is aimed at looking at all the attributes of the posting (e.g., posting date, rating, benefits, job title, logo, and company name), however, the structure of the site has divided these details in three bigger chuncks. So far i think it would be easier to go one detail at a time, at this point. Might come back later to try again. 

In [11]:
details = sel_soup.find_all('td',
                           attrs={'class':
                                   'offerslistRow'})

In [12]:
len(details)

45

In [22]:
test = details[0:4]
a =3 # Details
b =4 # Logo
c =5 # Company's Name
n = 3 # Sequence pattern
records = []
for i,x in enumerate(test):
    if (i+3)% n == 0:
        date = x.find('span',attrs={'class':'card__subtitle mdc-typography mdc-typography--overline line-height-1-5'}).text.strip()
        rating = x.find('span',attrs={'class':'iconed'}).text.strip()
        job = x.find('a',attrs={'class':'card__title mdc-typography mdc-typography--headline6 text-overflow'}).text.strip()
        benefits = x.find('span',attrs={'class':'card__subtitle mdc-typography mdc-typography--body2 top-margin'}).text.strip()
    elif 
    records.append((date, rating, job, benefits))

In [43]:
a = 
a+3*(a-1)

45

In [ ]:
det = [0]
for a in range(0,10):
    for i in det:
        z = i+3*(max(det)-1)
    det.append(z)

det    

In [23]:
records

[('днес',
  '\uf4b3\uf4b3\uf4b3\uf4b3\uf4b3',
  'Customer Support Specialist with English',
  'София'),
 ('днес',
  '\uf4b3\uf4b3\uf4b3\uf4b3\uf4b3',
  'Customer Support Specialist with English',
  'София'),
 ('днес',
  '\uf4b3\uf4b3\uf4b3\uf4b3\uf4b3',
  'Customer Support Specialist with English',
  'София'),
 ('днес',
  '\uf4b3\uf4b3\uf4b3\uf4b3\uf4b3',
  'Младши търговец в Снабдяване',
  'София')]

### Date Posted
<span class="card__subtitle mdc-typography mdc-typography--overline line-height-1-5" style="display:block;"> днес 
</span>

In [115]:
first_det = details[0]
first_det

<td class="offerslistRow" style="border-bottom:1px solid #cccccc;" valign="top">
<span class="card__subtitle mdc-typography mdc-typography--overline line-height-1-5" style="display:block;">
                                                                    днес                                                                                                                                                                                                                <span class="f-right light-blue icon" title="При тази позиция се предлага възможност за работа от вкъщи. За повече информация вижте обявата.">
<i class="icon-size-24">
<!--?xml version="1.0"?--><svg fill="#000000" height="96px" viewbox="0 0 24 24" width="96px" xmlns="http://www.w3.org/2000/svg"> <path d="M 12 2.0996094 L 1 12 L 4 12 L 4 21 L 12.294922 21 C 12.105922 20.366 12 19.695 12 19 L 6 19 L 6 10.189453 L 12 4.7910156 L 20.039062 12 L 23 12 L 12 2.0996094 z M 19.003906 13.988281 C 16.242906 13.988281 14.003906 16.22728

In [100]:
dates = first_det.find('span',attrs={'class':'card__subtitle mdc-typography mdc-typography--overline line-height-1-5'})
dates.text.strip()

'днес'

In [101]:
rating = first_det.find('span',attrs={'class':'iconed'})
rating.text.strip()

'\uf4b3\uf4b3\uf4b3\uf4b3\uf4b3'

In [102]:
job = first_det.find('a',attrs={'class':'card__title mdc-typography mdc-typography--headline6 text-overflow'})
job.text.strip()

'Senior Software Engineer - Backend'

In [103]:
benefits = first_det.find('span',attrs={'class':'card__subtitle mdc-typography mdc-typography--body2 top-margin'})
benefits.text.strip()

'София; Възможност за работа от вкъщи; Дистанционно интервю'

In [104]:
company = first_det.find('a',attrs={'class':'company_link'})
company.text.strip()

AttributeError: 'NoneType' object has no attribute 'text'

In [61]:
date_res = sel_soup.find_all('span',
                            attrs={'class':
                                   'card__subtitle mdc-typography mdc-typography--overline line-height-1-5'})

In [51]:
len(date_res) # Check if i get the right amount of dates

15

In [68]:
date_res[0] # check the first 3 (notice that it also includes some benefits for some of the postings)

<span class="card__subtitle mdc-typography mdc-typography--overline line-height-1-5" style="display:block;">
                                                                    днес                                                                                                                                                                                                                <span class="f-right light-blue icon" title="При тази позиция се предлага възможност за работа от вкъщи. За повече информация вижте обявата.">
<i class="icon-size-24">
<!--?xml version="1.0"?--><svg fill="#000000" height="96px" viewbox="0 0 24 24" width="96px" xmlns="http://www.w3.org/2000/svg"> <path d="M 12 2.0996094 L 1 12 L 4 12 L 4 21 L 12.294922 21 C 12.105922 20.366 12 19.695 12 19 L 6 19 L 6 10.189453 L 12 4.7910156 L 20.039062 12 L 23 12 L 12 2.0996094 z M 19.003906 13.988281 C 16.242906 13.988281 14.003906 16.227281 14.003906 18.988281 C 14.003906 21.749281 16.242906 23.988281 19.003906 23.9882

In [71]:
first_rec = date_res[0]
first_rec_text = first_rec.text
first_rec_text.strip()

'днес'

### Star Rating
<span class="iconed" style="color:#66c1ff;;vertical-align:text-top;letter-spacing: 1px;"></span>

The star rating that jobs.bg offers are corrupted. The browser is not showing the starts accordingly and python is returning "\uf4b3" rather than a symbol. Will check it out later.

In [18]:
star_res = sel_soup.find_all('span',
                            attrs={'class':
                                   'iconed'})

In [19]:
len(star_res)

15

In [20]:
star_res[0:3] # check the first 3 (notice that it also includes some benefits for some of the postings)

[<span class="iconed" style="color:#66c1ff;;vertical-align:text-top;letter-spacing: 1px;"></span>,
 <span class="iconed" style="color:#66c1ff;;vertical-align:text-top;letter-spacing: 1px;"></span>,
 <span class="iconed" style="color:#66c1ff;;vertical-align:text-top;letter-spacing: 1px;"></span>]

In [21]:
first_rec = star_res[0]
first_rec.contents

['\uf4b3\uf4b3\uf4b3\uf4b3\uf4b3']

### Title of the job posting
<a class="card__title mdc-typography mdc-typography--headline6 text-overflow" href="job/5641066">Специалист Tехническа Поддръжка</a>

In [22]:
title_res = sel_soup.find_all('a',
                            attrs={'class':
                                   'card__title mdc-typography mdc-typography--headline6 text-overflow'})

In [23]:
len(title_res)

15

In [24]:
title_res[0:3] # check the first 3 (notice that it also includes some benefits for some of the postings)

[<a class="card__title mdc-typography mdc-typography--headline6 text-overflow" href="job/5625722">Senior Software Engineer - Backend</a>,
 <a class="card__title mdc-typography mdc-typography--headline6 text-overflow" href="job/5641066">Специалист Tехническа Поддръжка</a>,
 <a class="card__title mdc-typography mdc-typography--headline6 text-overflow" href="job/5645027">Customer Support Specialist with English</a>]

In [25]:
first_rec = title_res[0]
first_rec.text

'Senior Software Engineer - Backend'

### Benefits/ Location
<div class="spacing">
<span class="card__subtitle mdc-typography mdc-typography--body2 top-margin">
София
</span>
</div>

In [26]:
benefit_res = sel_soup.find_all('span',
                            attrs={'class':
                                   'card__subtitle mdc-typography mdc-typography--body2 top-margin'})

In [27]:
len(benefit_res)

15

In [28]:
benefit_res[0:3] # check the first 3 (notice that it also includes some benefits for some of the postings)

[<span class="card__subtitle mdc-typography mdc-typography--body2 top-margin">
 
                                                                         София; Възможност за работа от вкъщи; Дистанционно интервю
                                                                     </span>,
 <span class="card__subtitle mdc-typography mdc-typography--body2 top-margin">
 
                                                                         София
                                                                     </span>,
 <span class="card__subtitle mdc-typography mdc-typography--body2 top-margin">
 
                                                                         София
                                                                     </span>]

In [29]:
first_rec = benefit_res[5]
first_rec_text = first_rec.text
first_rec_text.strip()

'София'

### Company Name
<a class="company_link" href="https://www.jobs.bg/company/166091" style="font-size:0.85em;line-height:1.5;">ТЕК ЕКСПЪРТС ЕООД</a>

In [30]:
name_res = sel_soup.find_all('a',
                            attrs={'class':
                                   'company_link'})

In [31]:
len(name_res)

15

In [32]:
name_res[0:3] # check the first 3 (notice that it also includes some benefits for some of the postings)

[<a class="company_link" href="https://www.jobs.bg/company/224220" style="font-size:0.85em;line-height:1.5;">HyperScience EOOD</a>,
 <a class="company_link" href="https://www.jobs.bg/company/166091" style="font-size:0.85em;line-height:1.5;">ТЕК ЕКСПЪРТС ЕООД</a>,
 <a class="company_link" href="https://www.jobs.bg/company/166091" style="font-size:0.85em;line-height:1.5;">ТЕК ЕКСПЪРТС ЕООД</a>]

In [33]:
first_rec = name_res[0]
first_rec_text = first_rec.text
first_rec_text.strip()

'HyperScience EOOD'

### Combining the data

In [34]:
records = []
for i in details:
    

SyntaxError: invalid syntax (<ipython-input-34-f2e6c63ea815>, line 2)